# Learning NLP using transformers

In [1]:
#hide
# Import time!
from loguru import logger
from rich.logging import RichHandler

# Logging setup
logger.configure(
    handlers=[
        {
            "sink": RichHandler(
                log_time_format="%Y-%m-%d %X",
                rich_tracebacks=True,
                show_path=False,
                tracebacks_width=200,
            ),
            "format": "{message}",
            "level": "DEBUG",
        }
    ]
)

[1]

In [2]:
from pathlib import Path

data_dir = Path('./us-patent-phrase-to-phrase-matching/')

We're creating a net to analyse the Kaggle Patent competition

Let's open the .csv first

In [24]:
import pandas as pd

df = pd.read_csv(data_dir / 'train.csv')
eval_df = pd.read_csv(data_dir / 'test.csv')

In [4]:
df

,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00
...,...,...,...,...,...
36468,8e1386cbefd7f245,wood article,wooden article,B44,1.00
36469,42d9e032d1cd3242,wood article,wooden box,B44,0.50
36470,208654ccb9e14fa3,wood article,wooden handle,B44,0.50
36471,756ec035e694722b,wood article,wooden material,B44,0.75


In [5]:
df.describe(include='object')

,id,anchor,target,context
count,36473,36473,36473,36473
unique,36473,733,29340,106
top,37d61fd2272659b1,component composite coating,composition,H01
freq,1,152,24,2186


In [29]:
eval_df['input'] = 'TEXT1: ' + eval_df.context + '; TEXT2: ' + eval_df.target + '; ANC1: ' + eval_df.anchor
df['input'] = 'TEXT1: ' + df.context + '; TEXT2: ' + df.target + '; ANC1: ' + df.anchor

In [7]:
df.input.head()

0    TEXT1: A47; TEXT2: abatement of pollution; ANC...
1    TEXT1: A47; TEXT2: act of abating; ANC1: abate...
2    TEXT1: A47; TEXT2: active catalyst; ANC1: abat...
3    TEXT1: A47; TEXT2: eliminating process; ANC1: ...
4    TEXT1: A47; TEXT2: forest region; ANC1: abatement
Name: input, dtype: object

After we've created the data table, we need to convert this into a dataset using tokenisation

In [31]:
from datasets import Dataset, DatasetDict

ds = Dataset.from_pandas(df)
eval_ds = Dataset.from_pandas(eval_df)
eval_ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'input'],
    num_rows: 36
})

With this, we need to perform tokenisation and numericalisation

In [9]:
model_nm = 'microsoft/deberta-v3-small'

In [10]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

tokz = AutoTokenizer.from_pretrained(model_nm)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/Users/kmccarro/miniconda3/envs/fast_ai/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:470: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
def tok_func(x): return tokz(x['input'])

In [32]:
tokenised_dataset = ds.map(tok_func, batched=True)
eval_ds = eval_ds.map(tok_func, batched=True)

Map:   0%|          | 0/36473 [00:00<?, ? examples/s]

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

In [13]:
row = tokenised_dataset[0]
row['input'], row['input_ids']

('TEXT1: A47; TEXT2: abatement of pollution; ANC1: abatement',
 [1,
  54453,
  435,
  294,
  336,
  5753,
  346,
  54453,
  445,
  294,
  47284,
  265,
  6435,
  346,
  23702,
  435,
  294,
  47284,
  2])

In [33]:
tokz.vocab['▁of']

265

In [15]:
tokenised_dataset = tokenised_dataset.rename_columns({'score': 'labels'})

In [16]:
import numpy as np
def corr(x, y): return np.corrcoef(x, y)[0][1]

def corr_d(eval_pred): return {'pearson': corr(*eval_pred)}

In [17]:
# Create the train_test split

dds = tokenised_dataset.train_test_split(0.25, seed=42)
dds

DatasetDict({
    train: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 27354
    })
    test: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9119
    })
})

Now after all that, we're ready to create and train our model

In [18]:
from transformers import TrainingArguments, Trainer

batch_size = 128
epoch_count = 4

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [19]:
learning_rate = 8e-5

In [21]:
args = TrainingArguments(
    'outputs',
    learning_rate=learning_rate,
    warmup_ratio=0.1,
    lr_scheduler_type='cosine',
    fp16=False,
    evaluation_strategy='epoch',
    per_device_train_batch_size=batch_size,
    per_gpu_eval_batch_size=batch_size*2,
    num_train_epochs=epoch_count,
    weight_decay=0.01,
    report_to='none'
)

In [22]:
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)
trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'], tokenizer=tokz, compute_metrics=corr_d)

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.weight', 'pooler.dense.bias', 'classifier.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
trainer.train()

  0%|          | 0/856 [00:00<?, ?it/s]

You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Using deprecated `--per_gpu_eval_batch_size` argument which will be removed in a future version. Using `--per_device_eval_batch_size` is preferred.
Using deprecated `--per_gpu_eval_batch_size` argument which will be removed in a future version. Using `--per_device_eval_batch_size` is preferred.


  0%|          | 0/36 [00:00<?, ?it/s]

Using deprecated `--per_gpu_eval_batch_size` argument which will be removed in a future version. Using `--per_device_eval_batch_size` is preferred.


{'eval_loss': 0.026670819148421288, 'eval_pearson': 0.7900483843516456, 'eval_runtime': 15.042, 'eval_samples_per_second': 606.237, 'eval_steps_per_second': 2.393, 'epoch': 1.0}


...

Using deprecated `--per_gpu_eval_batch_size` argument which will be removed in a future version. Using `--per_device_eval_batch_size` is preferred.
Using deprecated `--per_gpu_eval_batch_size` argument which will be removed in a future version. Using `--per_device_eval_batch_size` is preferred.


  0%|          | 0/36 [00:00<?, ?it/s]

Using deprecated `--per_gpu_eval_batch_size` argument which will be removed in a future version. Using `--per_device_eval_batch_size` is preferred.


{'eval_loss': 0.02333071082830429, 'eval_pearson': 0.8302300472754015, 'eval_runtime': 12.2257, 'eval_samples_per_second': 745.886, 'eval_steps_per_second': 2.945, 'epoch': 4.0}
{'train_runtime': 516.3023, 'train_samples_per_second': 211.922, 'train_steps_per_second': 1.658, 'train_loss': 0.0245761743215757, 'epoch': 4.0}


TrainOutput(global_step=856, training_loss=0.0245761743215757, metrics={'train_runtime': 516.3023, 'train_samples_per_second': 211.922, 'train_steps_per_second': 1.658, 'train_loss': 0.0245761743215757, 'epoch': 4.0})

In [34]:
test_predictions = trainer.predict(eval_ds).predictions.astype(float)
test_predictions

Using deprecated `--per_gpu_eval_batch_size` argument which will be removed in a future version. Using `--per_device_eval_batch_size` is preferred.
Using deprecated `--per_gpu_eval_batch_size` argument which will be removed in a future version. Using `--per_device_eval_batch_size` is preferred.


  0%|          | 0/1 [00:00<?, ?it/s]

Using deprecated `--per_gpu_eval_batch_size` argument which will be removed in a future version. Using `--per_device_eval_batch_size` is preferred.


array([[ 0.56808966],
       [ 0.66910297],
       [ 0.45617303],
       [ 0.28771564],
       [-0.04568528],
       [ 0.53055072],
       [ 0.55161148],
       [-0.04402247],
       [ 0.24500091],
       [ 1.13461685],
       [ 0.25275519],
       [ 0.22057894],
       [ 0.7660566 ],
       [ 0.89892894],
       [ 0.76206177],
       [ 0.44694871],
       [ 0.26779658],
       [-0.04828648],
       [ 0.66114527],
       [ 0.33773372],
       [ 0.39381084],
       [ 0.22454056],
       [ 0.03844251],
       [ 0.24399571],
       [ 0.56545514],
       [-0.03354459],
       [-0.03002143],
       [-0.04918579],
       [-0.03902717],
       [ 0.69954282],
       [ 0.26982036],
       [ 0.02547619],
       [ 0.70082605],
       [ 0.50316757],
       [ 0.43378517],
       [ 0.19080995]])

In [35]:
test_predictions = np.clip(test_predictions, 0, 1)
test_predictions

array([[0.56808966],
       [0.66910297],
       [0.45617303],
       [0.28771564],
       [0.        ],
       [0.53055072],
       [0.55161148],
       [0.        ],
       [0.24500091],
       [1.        ],
       [0.25275519],
       [0.22057894],
       [0.7660566 ],
       [0.89892894],
       [0.76206177],
       [0.44694871],
       [0.26779658],
       [0.        ],
       [0.66114527],
       [0.33773372],
       [0.39381084],
       [0.22454056],
       [0.03844251],
       [0.24399571],
       [0.56545514],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.69954282],
       [0.26982036],
       [0.02547619],
       [0.70082605],
       [0.50316757],
       [0.43378517],
       [0.19080995]])